In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import time
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
from torchvision.models import resnet50
from torchvision.datasets import ImageFolder
from torch import nn, optim
from torch.cuda.amp import autocast, GradScaler

# Path to the dataset
data_dir = '/content/drive/MyDrive/JPEGImages'

In [3]:
# 1. Data Loading and Preprocessing
def load_split_data(data_dir, is_train=True):
    """ Load and preprocess data, returning a DataLoader. """
    transform = {
        'train': transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(10),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'test': transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
    }

    dataset = ImageFolder(root=data_dir, transform=transform['train' if is_train else 'test'])
    if is_train:
        class_counts = [0] * 50
        for _, target in dataset.samples:
            class_counts[target] += 1
        class_weights = [1 / (count + 1e-6) for count in class_counts]
        sample_weights = [class_weights[target] for _, target in dataset.samples]
        sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)
        loader = DataLoader(dataset, batch_size=64, sampler=sampler, num_workers=8, pin_memory=True)
    else:
        loader = DataLoader(dataset, batch_size=64, shuffle=False, num_workers=8, pin_memory=True)

    return loader

train_loader = load_split_data(data_dir, is_train=True)
test_loader = load_split_data(data_dir, is_train=False)

In [4]:
# 2. Model Definition
def get_model(num_classes=50):
    model = resnet50(pretrained=True)
    num_features = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Linear(num_features, 512),
        nn.BatchNorm1d(512),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(512, num_classes)
    )
    return model

model = get_model()
model = model.cuda()



/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 152MB/s]


In [8]:
# 3. Training
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scaler = GradScaler()

def train_model():
    start_time = time.time()
    model.train()
    for epoch in range(10):
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.cuda(), labels.cuda()
            optimizer.zero_grad()
            with autocast():
                outputs = model(inputs)
                loss = criterion(outputs, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            if i % 20 == 0:
                print(f"Epoch {epoch}, Step {i}, Loss: {loss.item()}")

    end_time = time.time()
    total_time = end_time - start_time
    print(f"Total training time: {total_time:.2f} seconds")


In [9]:
# 4. Evaluation
def evaluate_model():
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.cuda(), labels.cuda()
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Accuracy on the test set: {100 * correct / total}%')



In [10]:
train_model()
evaluate_model()

Epoch 0, Step 0, Loss: 2.6421828269958496
Epoch 0, Step 20, Loss: 2.7917087078094482
Epoch 0, Step 40, Loss: 2.0624160766601562
Epoch 0, Step 60, Loss: 2.3145625591278076
Epoch 0, Step 80, Loss: 1.890323519706726
Epoch 0, Step 100, Loss: 1.4729232788085938
Epoch 0, Step 120, Loss: 1.4799585342407227
Epoch 0, Step 140, Loss: 1.6700005531311035
Epoch 0, Step 160, Loss: 1.5505094528198242
Epoch 1, Step 0, Loss: 1.6505286693572998
Epoch 1, Step 20, Loss: 1.671342134475708
Epoch 1, Step 40, Loss: 1.6563447713851929
Epoch 1, Step 60, Loss: 1.389883041381836
Epoch 1, Step 80, Loss: 1.5580463409423828
Epoch 1, Step 100, Loss: 1.309066891670227
Epoch 1, Step 120, Loss: 1.1400022506713867
Epoch 1, Step 140, Loss: 1.4351556301116943
Epoch 1, Step 160, Loss: 1.5998358726501465
Epoch 2, Step 0, Loss: 0.9827694892883301
Epoch 2, Step 20, Loss: 1.0731240510940552
Epoch 2, Step 40, Loss: 1.4540128707885742
Epoch 2, Step 60, Loss: 0.9684163331985474
Epoch 2, Step 80, Loss: 1.4796311855316162
Epoch 2, S